<a href="https://colab.research.google.com/github/Zheng-Ao/TechMining/blob/master/TechMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goals
See the picture.<br>
Y: 暂时就用我自定义的。 

# End2End Pipline

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Instrument 0: libraries

In [2]:
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"

1.11.0+cu113
11.3


In [3]:
!pip install transformers
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 26.9 MB/s 
     |████████████████████████████████| 596 kB 51.9 MB/s 
     |████████████████████████████████| 6.6 MB 48.2 MB/s 
     |████████████████████████████████| 101 kB 12.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 7.9 MB 37.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 3.5 MB 32.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/c

In [4]:
import pandas as pd
from datetime import datetime

import torch
import torch.nn.functional as F

from transformers import DistilBertTokenizer, DistilBertModel

from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

## I1: NLP+ for feature construction

In [5]:
DATA = pd.read_csv("drive/MyDrive/GT36.csv")

In [6]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
pn_to_id = {}

for item in DATA.values:
    pn_to_id[item[1]] = item[0]


In [8]:
now = datetime.strptime("2022-06-21", "%Y-%M-%d").year
pn_ls = [item[1] for item in DATA.values]

n_nodes = len(DATA)
node_dim = 2        # CPC, IPC


Nodes_x = []        # CPC, IPC
Nodes_txt = torch.empty(1, 768)
Nodes_y = []        # label
Edges = []          # conectivity weights
Adj_Ls = []         # conectivity

for i in range(len(DATA)):
    pn = DATA['PN'][i]
    ref = DATA['REF'][i]
    b_cits = eval(DATA['b_cits'][i])
    isd = DATA['ISD'][i]

    # x
    node = [DATA['CPC'][i], DATA['IPC'][i]]
    Nodes_x.append(node)

    # text
    ttl = DATA['TTL'][i].replace('\n', '')
    inputs = tokenizer(ttl, return_tensors="pt")
    outputs = model(**inputs)

    last_hidden_states = outputs.last_hidden_state

    cls_vec = last_hidden_states[:,0,:].clone().detach()

    if i == 0:
      Nodes_txt = cls_vec
    else:
      Nodes_txt = torch.cat([Nodes_txt, cls_vec], dim = 0)



    # y
    years = now - datetime.strptime(isd, "%Y-%M-%d").year
    score = ref/years       # Optimize it with python lambda
    if score > 1:
        node_y = 1          # positive
    else:
        node_y = 0          # negative
    Nodes_y.append(node_y)

    # conectivity
    if b_cits[0] == 'NULL' or b_cits[0] == '':
        continue
    for cp in b_cits:

        # 注意：cp 必须是已知节点，即在Nodes中 
        if cp.find('/') == -1:
            try:
                if int(cp) in pn_ls:
                    pn_id = pn_to_id[pn]
                    cp_id = pn_to_id[int(cp)]
                    Adj_Ls.append([pn_id, cp_id])
                    duration = (pn - int(cp))/1e7
                    Edges.append(duration)            # 边的权重

            except Exception as ex:
                pass
                # print("{}采集出错，出错原因：{}".format(pn, ex))

In [9]:
Nodes_txt.dtype

torch.float32

## I2: GNN

In [19]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(768, 16)
        self.conv2 = GCNConv(16, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [12]:
x = torch.tensor(Nodes_x, dtype=torch.float)
Nodes_x = torch.cat([x, Nodes_txt], dim=1)

In [13]:
Nodes_x.shape

torch.Size([564, 770])

In [20]:
# num_nodes, num_node_features
x = torch.tensor(Nodes_x, dtype=torch.float)

# num_nodes, 1
y = torch.tensor(Nodes_y, dtype=torch.long)

# 2, num_edges
edge_index = torch.tensor(Adj_Ls, dtype=torch.long)

# num_edges, num_edge_features
edge_attr = torch.tensor(Edges, dtype=torch.float)

# train-test split
train_ls = [True]*400 + [False]*164
train_mask = torch.tensor(train_ls, dtype=bool)
test_ls = [False]*400 + [True]*164
test_mask = torch.tensor(test_ls, dtype=bool)


data = Data(x=Nodes_txt, y=y, edge_index = edge_index.t().contiguous(), edge_attr=edge_attr, train_mask = train_mask, test_mask=test_mask)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [21]:
print(f'Number of nodes: {data.num_nodes}') # 节点数量
print(f'Number of edges: {data.num_edges}') # 边数量
print(f'Number of node features: {data.num_node_features}') # 节点属性的维度
print(f'Number of node features: {data.num_features}') # 同样是节点属性的维度
print(f'Number of edge features: {data.num_edge_features}') # 边属性的维度
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}') # 平均节点度
# print(f'if edge indices are ordered and do not contain duplicate entries.: {data.is_coalesced()}') # 是否边是有序的同时不含有重复的边
print(f'Number of training nodes: {data.train_mask.sum()}') # 用作训练集的节点
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}') # 用作训练集的节点的数量
print(f'Contains isolated nodes: {data.has_isolated_nodes()}') # 此图是否包含孤立的节点
print(f'Contains self-loops: {data.has_self_loops()}')  # 此图是否包含自环的边
print(f'Is undirected: {data.is_undirected()}')  # 此图是否是无向图

Number of nodes: 564
Number of edges: 687
Number of node features: 768
Number of node features: 768
Number of edge features: 1
Average node degree: 1.22
Number of training nodes: 400
Training node label rate: 0.71
Contains isolated nodes: True
Contains self-loops: False
Is undirected: False


In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
g_model = GCN().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = g_model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [23]:
model.eval()
pred = g_model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.6220


# Bottlenecks, Findings in conclusion
1. Lack of enough Data.(1e6 needed, now 1e3).<br>
2. Lack of background information.(The model can not learn to judge patents just from prior patents. Other scientific knowledge and industry knowledge is needed)